# generating normal log files for checking error

In [ ]:
from pyspark import SparkContext, SparkConf #spark

In [2]:
import pydoop.hdfs as hdfs #for saving local file as hdfs 

In [17]:
from pyspark.sql import SQLContext # for data frame

In [87]:
import urllib #for imitating hdfs auto escape string

In [ ]:
import re #for escape string 

In [3]:
conf = SparkConf().setAppName("error_machine")

In [4]:
sc = SparkContext(conf=conf)

# upload file to hdfs

##### check result:
* [hdfs@ambari kristen]$ hdfs dfs -ls /user/spark/input
*  Found 2 items
*  -rw-r--r--   3 root root     213152 2017-03-08 05:06 /user/spark/input/log_error
*  -rw-r--r--   3 root root    1363076 2017-03-08 05:04 /user/spark/input/log_normal


In [5]:
from_path = './log_normal'

In [6]:
to_path ='hdfs://ambari:8020/user/spark/input'

In [7]:
hdfs.put(from_path, to_path)

IOError: '/user/spark/input/log_normal' already exists

In [ ]:
from_path = './log_error'

In [ ]:
to_path ='hdfs://ambari:8020/user/spark/input'

In [ ]:
hdfs.put(from_path, to_path)

# classificat normal log into multiple folder

In [208]:
def Group_By_Feature(df):
    '''
    Grouping main feature of each log text
    '''
    key = "none"
    value = "none"
    if (len(df.split(" ")) >5):
        if ("[" in df.split(" ")[5]):
            key = df.split(" ")[5]
            key = key.split("[")[0]
            value = df.split(df.split(" ")[5])[1]
    return (str(key).lower(),str(value))

In [ ]:
def MapReduce_Normal_Log(inputPath, outputFolder):
    '''
    1. grouping each text lines by main keys, and generating main key files.
    2. looping mapreduce for distinct each content by main key files.
    '''
    #1 run: map reduce (grouping by main keys)
    in_hdfs = sc.textFile(inputPath)
    in_map = in_hdfs.map(lambda s : Group_By_Feature(s))
    res_reduce = in_map.reduceByKey(lambda a, b: a+'\n'+b + '\n' )
    
    #convert rdd -> data frame
    df = res_reduce.toDF(["keyword"])
    
    #saving multiple files by keyword (file name = keyword name )
    df.write.partitionBy("keyword").text(outputFolder)
    
    #2 run: map reduct (to be distinct each content)
    for keyword in collect_df_key:
        keyword = str(keyword[0])
        # distinct values in each keyword files [urllib.quote_plus: convert / -> %x]
        key_hdfs = sc.textFile(outputFolder+"/keyword="+ urllib.quote_plus(keyword).lower()) #import urllib
        key_map = key_hdfs.map(lambda s : (s,1))
        key_res = key_map.reduceByKey(lambda a, b: a+b)
        key_res.saveAsTextFile(outputFolder.split("/normal_1")[0]+"/"+"log_distinct"+"/"+keyword)

In [ ]:
inputPath = '/user/spark/input/log_normal'

In [ ]:
outputFolder = '/user/spark/output/'

In [ ]:
MapReduce_Normal_Log(inputPath, outputFolder)

# try:　MapReduce Normal Log

In [209]:
inputPath = '/user/spark/input/log_normal'

In [210]:
in_hdfs = sc.textFile(inputPath)

In [211]:
in_map = in_hdfs.map(lambda s : Group_By_Feature(s))

In [212]:
in_map.take(1000)

[('anacron', " Job `cron.daily' terminated"),
 ('anacron', ' Normal exit (1 job run)'),
 ('dhclient',
  ' DHCPREQUEST of 192.168.146.129 on ens33 to 192.168.146.254 port 67 (xid=0x3da2d28b)'),
 ('dhclient', ' DHCPACK of 192.168.146.129 from 192.168.146.254'),
 ('networkmanager', ' <info>  [1483573463.2670]   address 192.168.146.129'),
 ('networkmanager', ' <info>  [1483573463.2671]   plen 24 (255.255.255.0)'),
 ('networkmanager', ' <info>  [1483573463.2672]   gateway 192.168.146.2'),
 ('networkmanager',
  ' <info>  [1483573463.2672]   server identifier 192.168.146.254'),
 ('dbus',
  " [system] Activating via systemd: service name='org.freedesktop.nm_dispatcher' unit='dbus-org.freedesktop.nm-dispatcher.service'"),
 ('networkmanager', ' <info>  [1483573463.2672]   lease time 1800'),
 ('dhclient', ' bound to 192.168.146.129 -- renewal in 719 seconds.'),
 ('networkmanager', " <info>  [1483573463.2673]   nameserver '192.168.146.2'"),
 ('networkmanager', " <info>  [1483573463.2673]   domain 

In [213]:
df_map = in_map.toDF(['key'])

In [214]:
df_map[['key']]

DataFrame[key: string]

In [124]:
df_map[['key']].take(32)

[Row(key=u'anacron'),
 Row(key=u'anacron'),
 Row(key=u'dhclient'),
 Row(key=u'dhclient'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'dbus'),
 Row(key=u'NetworkManager'),
 Row(key=u'dhclient'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'systemd'),
 Row(key=u'dbus'),
 Row(key=u'systemd'),
 Row(key=u'none'),
 Row(key=u'none'),
 Row(key=u'systemd'),
 Row(key=u'dhclient'),
 Row(key=u'dhclient'),
 Row(key=u'dhclient'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager'),
 Row(key=u'NetworkManager')]

In [215]:
res_reduce = in_map.reduceByKey(lambda a, b: a+'\n'+b + '\n' )

In [216]:
res_reduce.take(2)

[('systemd',
  " Starting Network Manager Script Dispatcher Service...\n Started Network Manager Script Dispatcher Service.\n\n Started CUPS Scheduler.\n\n Starting Network Manager Script Dispatcher Service...\n\n Started Network Manager Script Dispatcher Service.\n\n Started CUPS Scheduler.\n\n Starting Network Manager Script Dispatcher Service...\n\n Started Network Manager Script Dispatcher Service.\n\n Started CUPS Scheduler.\n\n Starting Network Manager Script Dispatcher Service...\n\n Started Network Manager Script Dispatcher Service.\n\n Started CUPS Scheduler.\n\n Starting Network Manager Script Dispatcher Service...\n\n Started Network Manager Script Dispatcher Service.\n\n Started CUPS Scheduler.\n\n Starting Network Manager Script Dispatcher Service...\n\n Started Network Manager Script Dispatcher Service.\n\n Started CUPS Scheduler.\n\n Starting Network Manager Script Dispatcher Service...\n\n Started Network Manager Script Dispatcher Service.\n\n Started CUPS Scheduler.\n\

In [217]:
df = res_reduce.toDF(['keyword'])

In [218]:
df.show(3)

+------------------+--------------------+
|           keyword|                  _2|
+------------------+--------------------+
|           systemd| Starting Network...|
|org.gnome.terminal| (gnome-terminal-...|
|          whoopsie| [11:19:58] Canno...|
+------------------+--------------------+
only showing top 3 rows



In [219]:
outputFolder = "/user/spark/output/normal_1"

In [220]:
df.write.partitionBy("keyword").text(outputFolder)

### res: saving files to hdfs
#### [hdfs@ambari kristen]$ hdfs dfs -ls  /user/spark/output/normal_1/
* Found 24 items
* -rw-r--r--   3 root root          0 2017-03-09 01:06 /user/spark/output/normal_1/_SUCCESS
* drwxr-xr-x   - root root          0 2017-03-09 01:06 /user/spark/output/normal_1/keyword=%2fusr%2flib%2fsnapd%2fsnapd
* drwxr-xr-x   - root root          0 2017-03-09 01:06 /user/spark/output/normal_1/keyword=anacron
* drwxr-xr-x   - root root          0 2017-03-09 01:06 /user/spark/output/normal_1/keyword=avahi-daemon
* drwxr-xr-x   - root root          0 2017-03-09 01:06 /user/spark/output/normal_1/keyword=cron
* drwxr-xr-x   - root root          0 2017-03-09 01:06 /user/spark/output/normal_1/keyword=dbus
* drwxr-xr-x   - root root          0 2017-03-09 01:06 /user/spark/output/normal_1/keyword=dhclient
* drwxr-xr-x   - root root          0 2017-03-09 01:06 /user/spark/output/normal_1/keyword=dnsmasq
* drwxr-xr-x   - root root          0 2017-03-09 01:06 /user/spark/output/normal_1/keyword=dockerd
*....


In [221]:
collect_df_key = df.collect()

In [222]:
num_df_key = len(collect_df_key)

In [223]:
print num_df_key

23


In [224]:
print outputFolder.split("/normal_1")[0]+"/"+"/log_distinct"+"/"+urllib.quote_plus("ssss")

/user/spark/output//log_distinct/ssss


In [225]:
for keyword in collect_df_key:
    keyword = str(keyword[0])
    print keyword

systemd
org.gnome.terminal
whoopsie
none
org.gnome.evolution.dataserver.sources5
cron
snap
libvirtd
gnome-session
avahi-daemon
dnsmasq
smbd
dbus
systemd-tmpfiles
systemd-timesyncd
dhclient
systemd-udevd
rtkit-daemon
anacron
gnome-session-binary
/usr/lib/snapd/snapd
dockerd
networkmanager


In [226]:
print urllib.quote_plus('/user/spark/output')

%2Fuser%2Fspark%2Foutput


In [227]:
for keyword in collect_df_key:
    keyword = str(keyword[0])
    print keyword
    # distinct values in each keyword files [urllib.quote_plus: convert / -> %x]
    key_hdfs = sc.textFile(outputFolder+"/keyword="+ urllib.quote_plus(keyword).lower()) #import urllib
    key_map = key_hdfs.map(lambda s : (s,1))
    key_res = key_map.reduceByKey(lambda a, b: a+b)
    key_res.saveAsTextFile(outputFolder.split("/normal_1")[0]+"/"+"log_distinct"+"/"+keyword)

systemd
org.gnome.terminal
whoopsie
none
org.gnome.evolution.dataserver.sources5
cron
snap
libvirtd
gnome-session
avahi-daemon
dnsmasq
smbd
dbus
systemd-tmpfiles
systemd-timesyncd
dhclient
systemd-udevd
rtkit-daemon
anacron
gnome-session-binary
/usr/lib/snapd/snapd
dockerd
networkmanager


In [132]:
df_key = df[["keyword"]]

In [133]:
df_key.show()

+--------------------+
|             keyword|
+--------------------+
|      systemd\-udevd|
|             systemd|
|  systemd\-timesyncd|
|org\.gnome\.Terminal|
|            whoopsie|
|org\.gnome\.evolu...|
|                none|
|                CRON|
|       rtkit\-daemon|
|            libvirtd|
|                snap|
|             dnsmasq|
|                smbd|
|                dbus|
|            dhclient|
|\/usr\/lib\/snapd...|
|       avahi\-daemon|
|      gnome\-session|
|gnome\-session\-b...|
|             anacron|
+--------------------+
only showing top 20 rows



In [64]:
keys = df_key
for i in keys:
    print i

Column<keyword>


In [67]:
keyword = ".usr.lib/snapd/snapd"
print urllib.quote_plus(keyword)
    

.usr.lib%2Fsnapd%2Fsnapd
